In [1]:
import cellCnn
import importlib

importlib.reload(cellCnn)
import random
import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt

from cellCnn.ms.utils.helpers import get_min_mean_from_clusters, get_chunks
from cellCnn.utils import mkdir_p
from cellCnn.plotting import plot_results
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df

import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [2]:
#%reset

In [3]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input.nosync'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840 # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}
outfolder = 'mtl_1'
mkdir_p(outfolder)

In [4]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [5]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [6]:
#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 
DONE


In [7]:
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_77283/30090753.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [8]:
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *


# batch_sizes = [40,80,120, 160]
batch_size_dict = dict()
batch_sizes = [120]
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd8 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           batch_size=batch_size)
    selection_pool_nindc_cd8 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           batch_size=batch_size)

    # make sure list are equally long:
    if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
        selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
    elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
        selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]

    all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    freqs = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict[batch_size] = (X, freqs, Y)

print('DONE: batches created')

DONE: batches created


In [9]:
### CD8 Freq regression task + MTL
model_container_cd8 = []
stats_dict_reg_cd8 = dict()
outdir_cd8 = 'mtl/cd8/'
mkdir_p(outdir_cd8)
mkdir_p(outdir_cd8+'models')

In [25]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    X = np.array([*X_train, *X_valid])
    cd8 = np.array([*cd8_train, *cd8_valid])
    y = np.array([*y_train, *y_valid])
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_reg_cd8_{batch_size}_stl'

    # todo nsubset parameter maybe different
    model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [cd8_valid, y_valid],
                             nsubset=1000,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=30, learning_rate=None,
                             ncell=200,
                             per_sample=True, regression=True,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=2)
    model_container_cd8.append(model)
print('DONE building models')

KeyboardInterrupt: 

In [24]:
mses = []

# what predict does is: Iterating over the best 3 nets and return an array per net with the predictions for all samples!
bs_idx = 1
for i, model in enumerate(model_container_cd8):
    stats_file = open(f"{outdir_cd8}/stats_reg_{model.ncell}_{model.nsubset}.txt", "w+")
    stats_file.write(f"Model {i}; Batchsize: {batch_sizes[bs_idx-1]}\n")
    if bs_idx % kf_split == 0: ## when the kf split amount is reaches the next batchsize is applied
        bs_idx = bs_idx +1
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    mse_test = mean_squared_error(y_test, test_pred[0])
    mse_train = mean_squared_error(y_train, train_pred[0])
    mse_valid = mean_squared_error(y_valid, valid_pred[0])
    mses.append(mse_test)

    stats_file.write('Desease: \n')
    stats_file.write(f'MSE y test {mse_test}\n')
    stats_file.write(f'MSE y train {mse_train}\n')
    stats_file.write(f'MSE y valid {mse_valid}\n')

    mse_test_cd8 = mean_squared_error(cd8_test, test_pred[1])
    mse_train_cd8 = mean_squared_error(cd8_train, train_pred[1])
    mse_valid_cd8 = mean_squared_error(cd8_valid, valid_pred[1])
    mses.append(mse_test_cd8)

    stats_file.write('Cd8: \n')
    stats_file.write(f'MSE y test {mse_test_cd8}\n')
    stats_file.write(f'MSE y train {mse_train_cd8}\n')
    stats_file.write(f'MSE y valid {mse_valid_cd8}\n')
    stats_file.write('\n')
    stats_file.close()
print('DONE')

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature n

/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature n

DONE
